# 分布式 Logistic 回归模型

### Introduction to Logistics Regression

(online.stat.psu.edu/stat508/lesson/9/9.1/9.1.2)

- 最终的目的是知道$Y$的分布，而这里的$Y$是一个0-1变量，即服从一个Bernoulli分布
- 因此Logit回归的本质是对$Y$的Bernoulli分布的参数的建模
- $P(Y=1)=p$
- 如果$Y_1,...,Y_n$是独立同分布的，则只有一个参数$p$；实际上每一个obs都有一个各自的参数
- 这里就需要自变量$X$的帮助，也就是$Y$，或者是对应的$p$应该是依赖于观测$X_i$的
- 为了量化这种影响关系，可以认为$p_i = x_i^T \beta$
- 但是这里的问题是左侧的$p_i$是0,1区间的，而右侧的为$\R$上取值
- 中间的桥梁即是input正负无穷区间，输出01区间 ！！分布函数！！
- Sigmoid: $\rho(x) = \frac{1}{1+e^{-x}}$

$$Y|x \sim Bernoulli(\rho(\beta^T x))$$

- 然而事实上这里的函数并不一定局限于Sigmoid！
- $\rho(\beta^T x)$ 表示$Y$取1的概率
- 这里便引出了**极大似然函数**，因其优异的估计性质，构造损失函数！【详细了解极大似然！！】
- $P(Y_i = y) = p_i^y(1-p_i)^y,~ y = 0~ \text{or}~ 1$
- $l = \sum \log P(Y_i =y_i) = \sum[ y_i \log p_i + ( 1 - y_i) \log (1 - p_i)]$, where $p_i = \rho(x_i \beta)$
- 通常设定目标损失函数为$L(\beta) = -l$，以求极小值的优化
- logistic回归的系数估计基本只有数值解，没有解析解，故需要通过迭代优化算法进行求解

## 优化算法
- 优化问题：$min_x f(x)$
  - 根据不同的$f$的性质由不同的优化求解策略
- 称为具有导数的函数为光滑函数，对于光滑函数的优化算法：
  - 一阶算法（e.g. 梯度下降法）
  - 二阶算法（e.g. 牛顿法）

### 梯度下降法
$$x := x - \alpha \frac{\partial f}{\partial x}|_{{x=x_{old}}}$$

- 称$\alpha$为步长/学习率

### 牛顿法
$$x := x - \alpha (\frac{\partial^2 f}{\partial x\partial x'})^{-1}\frac{\partial f}{\partial x}|_{{x=x_{old}}}$$
- 上述求逆部分为Hessian矩阵
- 通常可以固定$\alpha=1$，牛顿法可以“自行调整”步长

## logit 算法推导
- 一阶导数：【试自证！】
$$\partial L/\partial \beta = X'(\rho-y)$$ 
- 二阶导数
  $$\partial^2 L/\partial \beta\partial \beta' = X'WX, \\W=diag(\rho_1(1-\rho_1)\cdots\rho_n(1-\rho_n))$$

- Recall: `solve(A,b)`,   $A:n\times p ,b:n\times 1$ 时间复杂度：$O(n^3)$

## 代码操作

### 1. 准备工作

配置和启动 PySpark：

In [2]:
import findspark
findspark.init("/Users/xinby/Library/Spark")

from pyspark.sql import SparkSession
# 本地模式
spark = SparkSession.builder.\
    master("local[*]").\
    appName("Logistic Regression").\
    getOrCreate()
sc = spark.sparkContext
sc.setLogLevel("ERROR")
print(spark)
print(sc)

<SparkContext master=local[*] appName=Logistic Regression>


假设 $n\gg p$，利用 Numpy 生成模拟数据，并写入文件。

In [3]:
import os
import numpy as np
from scipy.special import expit, logit
np.set_printoptions(linewidth=100)

np.random.seed(123)
n = 100000
p = 100
x = np.random.normal(size=(n, p))
beta = np.random.normal(size=p)
prob = expit(x.dot(beta))  # p = 1 / (1 + exp(-x * beta))
y = np.random.binomial(1, prob, size=n)
dat = np.hstack((y.reshape(n, 1), x))
if not os.path.exists("data"):
    os.makedirs("data", exist_ok=True)
np.savetxt("data/logistic.txt", dat, fmt="%f", delimiter="\t")

其中 `expit()` 即 Sigmoid 函数，表达式为 $\rho(x)=1/(1+e^{-x})$。 

PySpark 读取文件并进行一些简单操作：

In [4]:
file = sc.textFile("data/logistic.txt")

# 打印矩阵行数
print(file.count())

# 空行
print()

# 打印前5行，并将每行字符串截尾
text = file.map(lambda x: x[:70] + "...").take(5)
print(*text, sep="\n")

100000

0.000000	-1.085631	0.997345	0.282978	-1.506295	-0.578600	1.651437	-2.4...
1.000000	0.642055	-1.977888	0.712265	2.598304	-0.024626	0.034142	0.179...
0.000000	0.703310	-0.598105	2.200702	0.688297	-0.006307	-0.206662	-0.0...
1.000000	0.765055	-0.828989	-0.659151	0.611124	-0.144013	1.316606	-0.7...
0.000000	1.534090	-0.529914	-0.490972	-1.309165	-0.008660	0.976813	-1....


### 2. 牛顿法迭代

Logistic 回归的系数估计没有显式解，但可以利用牛顿法迭代。参见 [https://online.stat.psu.edu/stat508/lesson/9/9.1/9.1.2](https://online.stat.psu.edu/stat508/lesson/9/9.1/9.1.2)。

迭代公式为 $\beta^{new}=(X'WX)^{-1}X'Wz$，其中 $z=X\beta^{old}+W^{-1}(y-prob)$，$prob$ 是 $\rho(X\beta^{old})$ 组成的向量，$W$ 是以 $prob\cdot (1-prob)$ 为对角线元素的对角矩阵。

当 $n\gg p$ 且 $p$ 不太大时，$X'WX$ 为 $p\times p$ 矩阵，$X'Wz$ 为 $p\times 1$ 向量，均可放入内存。因此，此时问题的核心在于计算 $X'WX$ 与 $X'Wz$。

对于 $X,W,z$  进行分块，则有 $X'Wz=\sum X_iW_iz_i$

首先进行分区映射：

In [5]:
# file = file.repartition(10)
print(file.getNumPartitions())

3


In [6]:
# str => np.array
def str_to_vec(line):
    # 分割字符串
    str_vec = line.split("\t")
    # 将每一个元素从字符串变成数值型
    num_vec = map(lambda s: float(s), str_vec)
    # 创建 Numpy 向量
    return np.fromiter(num_vec, dtype=float)

# Iter[str] => Iter[matrix]
def part_to_mat(iterator):
    # Iter[str] => Iter[np.array]
    iter_arr = map(str_to_vec, iterator)

    # Iter[np.array] => list(np.array)
    dat = list(iter_arr)

    # list(np.array) => matrix
    if len(dat) < 1:  # Test zero iterator
        mat = np.array([])
    else:
        mat = np.vstack(dat)

    # matrix => Iter[matrix]
    yield mat

In [7]:
dat = file.mapPartitions(part_to_mat).filter(lambda x: x.shape[0] > 0)
print(dat.count())

3


In [8]:
dat.first()

array([[ 0.000000e+00, -1.085631e+00,  9.973450e-01, ..., -1.363472e+00,  3.794010e-01,
        -3.791760e-01],
       [ 1.000000e+00,  6.420550e-01, -1.977888e+00, ..., -1.108510e-01, -3.412620e-01,
        -2.179460e-01],
       [ 0.000000e+00,  7.033100e-01, -5.981050e-01, ...,  4.156950e-01,  1.605440e-01,
         8.197610e-01],
       ...,
       [ 0.000000e+00,  7.020540e-01,  8.005120e-01, ...,  1.232969e+00, -1.771340e-01,
        -5.306110e-01],
       [ 1.000000e+00,  4.353180e-01, -1.903069e+00, ...,  3.697810e-01,  9.449400e-01,
         1.347800e+00],
       [ 0.000000e+00, -1.460545e+00, -2.886790e-01, ..., -1.800490e-01,  2.074455e+00,
         1.191000e-03]])

注意此时每个分区上的数据同时包含了因变量 $y$ 和自变量 $X$。给定当前估计 $\beta^{old}$，计算每个分区上的统计量 $X'WX$ 和 $X'Wz$：

In [9]:
def compute_stats(part_mat, beta_old):
    # 提取 X 和 y
    y = part_mat[:, 0]
    x = part_mat[:, 1:]
    # X * beta
    xb = x.dot(beta_old)
    # rho(X * beta)
    prob = expit(xb)
    # W 的对角线元素
    w = prob * (1.0 - prob) + 1e-6 #10^-6避免后续计算分母为0
    # X'W，数组广播操作，避免生成完整的 W
    xtw = x.transpose() * w
    # X'WX
    xtwx = xtw.dot(x)
    # X'Wz
    z = xb + (y - prob) / w
    xtwz = xtw.dot(z)
    return xtwx, xtwz

主循环：

In [10]:
import time

# 根据数据动态获取维度，不要使用之前模拟时的变量
p = dat.first().shape[1] - 1
# beta 初始化为 0 向量
beta_hat = np.zeros(p)

# 最大迭代次数
maxit = 30
# 收敛条件
eps = 1e-6

t1 = time.time()
for i in range(maxit):
    # 完整数据的 X'WX 和 X'Wz 是各分区的加和
    xtwx, xtwz = dat.map(lambda part: compute_stats(part, beta_hat)).reduce(lambda x, y: (x[0] + y[0], x[1] + y[1]))
    # 计算新 beta
    beta_new = np.linalg.solve(xtwx, xtwz)
    # 计算 beta 的变化
    resid = np.linalg.norm(beta_new - beta_hat)
    print(f"Iteration {i}, resid = {resid}")
    # 如果 beta 几乎不再变化，退出循环
    if resid < eps:
        break
    # 更新 beta
    beta_hat = beta_new
t2 = time.time()
print(f"\nfinished in {t2 - t1} seconds")

Iteration 0, resid = 1.5704037438983014


Iteration 1, resid = 1.3912432651305584


Iteration 2, resid = 1.7393341248434817


Iteration 3, resid = 2.082759636695286


Iteration 4, resid = 2.0636707533919343


Iteration 5, resid = 1.320062748258957


Iteration 6, resid = 0.351657288363312


Iteration 7, resid = 0.018832519429043803


Iteration 8, resid = 6.66925409159852e-05


Iteration 9, resid = 6.386267762136753e-08

finished in 20.349837064743042 seconds


关闭 Spark 连接：

In [ ]:
sc.stop()